# Rozdział 7: Tworzenie aplikacji czatowych
## Szybki start z Github Models API

Ten notatnik został zaadaptowany z [Repozytorium Przykładów Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), które zawiera notatniki umożliwiające dostęp do usług [Azure OpenAI](notebook-azure-openai.ipynb).


# Przegląd  
"Duże modele językowe to funkcje, które zamieniają tekst na tekst. Po otrzymaniu wejściowego ciągu tekstu, duży model językowy stara się przewidzieć, jaki tekst pojawi się dalej"(1). Ten notatnik "szybki start" wprowadzi użytkowników w podstawowe pojęcia związane z LLM, kluczowe wymagania pakietów potrzebnych do rozpoczęcia pracy z AML, da łagodne wprowadzenie do projektowania promptów oraz pokaże kilka krótkich przykładów różnych zastosowań.


## Spis treści  

[Przegląd](../../../../07-building-chat-applications/python)  
[Jak korzystać z usługi OpenAI](../../../../07-building-chat-applications/python)  
[1. Tworzenie własnej usługi OpenAI](../../../../07-building-chat-applications/python)  
[2. Instalacja](../../../../07-building-chat-applications/python)    
[3. Dane uwierzytelniające](../../../../07-building-chat-applications/python)  

[Przykłady użycia](../../../../07-building-chat-applications/python)    
[1. Podsumowanie tekstu](../../../../07-building-chat-applications/python)  
[2. Klasyfikacja tekstu](../../../../07-building-chat-applications/python)  
[3. Generowanie nowych nazw produktów](../../../../07-building-chat-applications/python)  
[4. Dostosowanie klasyfikatora](../../../../07-building-chat-applications/python)  

[Odnośniki](../../../../07-building-chat-applications/python)


### Zbuduj swój pierwszy prompt  
To krótkie ćwiczenie wprowadzi Cię w podstawy wysyłania promptów do modelu w Github Models na przykładzie prostego zadania "podsumowanie".


**Kroki**:  
1. Zainstaluj bibliotekę `azure-ai-inference` w swoim środowisku Pythona, jeśli jeszcze tego nie zrobiłeś.  
2. Załaduj standardowe biblioteki pomocnicze i skonfiguruj poświadczenia do Github Models.  
3. Wybierz model odpowiedni do Twojego zadania  
4. Stwórz prosty prompt dla modelu  
5. Wyślij swoje zapytanie do API modelu!


### 1. Zainstaluj `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Wybór odpowiedniego modelu  
Modele GPT-3.5-turbo lub GPT-4 potrafią rozumieć i generować naturalny język.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Projektowanie promptów  

„Magia dużych modeli językowych polega na tym, że poprzez uczenie się minimalizowania błędów przewidywania na ogromnych ilościach tekstu, modele te ostatecznie przyswajają sobie pojęcia przydatne do tych przewidywań. Na przykład uczą się takich rzeczy jak”(1):

* jak poprawnie pisać
* jak działa gramatyka
* jak parafrazować
* jak odpowiadać na pytania
* jak prowadzić rozmowę
* jak pisać w wielu językach
* jak programować
* itd.

#### Jak sterować dużym modelem językowym  
„Spośród wszystkich danych wejściowych do dużego modelu językowego, zdecydowanie największy wpływ ma tekstowy prompt”(1).

Duże modele językowe można nakłonić do generowania odpowiedzi na kilka sposobów:

Instrukcja: Powiedz modelowi, czego oczekujesz  
Uzupełnienie: Skłoń model do dokończenia rozpoczętego tekstu  
Demonstracja: Pokaż modelowi, czego chcesz, poprzez:  
Kilka przykładów w promptcie  
Setki lub tysiące przykładów w zbiorze danych do fine-tuningu

#### Istnieją trzy podstawowe zasady tworzenia promptów:

**Pokaż i powiedz**. Jasno określ, czego oczekujesz – poprzez instrukcje, przykłady lub ich połączenie. Jeśli chcesz, by model posortował listę alfabetycznie lub sklasyfikował akapit pod względem sentymentu, pokaż mu, o co chodzi.

**Dostarcz dobre dane**. Jeśli chcesz zbudować klasyfikator lub nauczyć model podążać za wzorcem, upewnij się, że masz wystarczająco dużo przykładów. Sprawdź dokładnie swoje przykłady – model zwykle jest na tyle „inteligentny”, by przeoczyć drobne błędy ortograficzne i udzielić odpowiedzi, ale może też uznać je za celowe, co wpłynie na rezultat.

**Sprawdź ustawienia.** Parametry temperature i top_p decydują o tym, jak bardzo model jest zdeterminowany w generowaniu odpowiedzi. Jeśli oczekujesz jednej poprawnej odpowiedzi, ustaw je niżej. Jeśli zależy Ci na większej różnorodności odpowiedzi, możesz je podnieść. Najczęstszy błąd to mylenie tych ustawień z „inteligencją” lub „kreatywnością” modelu.

Źródło: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Podsumuj tekst  
#### Wyzwanie  
Podsumuj tekst, dodając na końcu fragmentu „tl;dr:”. Zwróć uwagę, jak model potrafi wykonać wiele zadań bez dodatkowych instrukcji. Możesz eksperymentować z bardziej opisowymi poleceniami niż tl;dr, aby zmienić zachowanie modelu i dostosować otrzymywane podsumowania(3).  

Najnowsze badania wykazały znaczne postępy w wielu zadaniach i benchmarkach NLP dzięki wstępnemu trenowaniu na dużych zbiorach tekstów, a następnie dostrajaniu pod konkretne zadanie. Chociaż architektura jest zazwyczaj niezależna od zadania, ta metoda nadal wymaga specjalnych zbiorów danych do dostrajania, liczących tysiące lub dziesiątki tysięcy przykładów. Dla porównania, ludzie potrafią zazwyczaj wykonać nowe zadanie językowe na podstawie zaledwie kilku przykładów lub prostych instrukcji – z czym obecne systemy NLP wciąż mają trudności. Pokazujemy tutaj, że zwiększanie rozmiaru modeli językowych znacznie poprawia ich uniwersalność i skuteczność w zadaniach z niewielką liczbą przykładów, czasem osiągając poziom konkurencyjny wobec wcześniejszych, zaawansowanych metod dostrajania. 



Tl;dr


# Ćwiczenia dla różnych zastosowań  
1. Podsumuj tekst  
2. Sklasyfikuj tekst  
3. Wymyśl nowe nazwy produktów


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klasyfikacja tekstu  
#### Wyzwanie  
Przypisz elementy do kategorii podanych w czasie wnioskowania. W poniższym przykładzie zarówno kategorie, jak i tekst do sklasyfikowania, są podane w poleceniu (*playground_reference).

Zapytanie klienta: Dzień dobry, jeden z klawiszy na mojej klawiaturze laptopa niedawno się zepsuł i będę potrzebować zamiennika:

Przypisana kategoria:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Generuj nowe nazwy produktów
#### Wyzwanie
Stwórz nazwy produktów na podstawie przykładowych słów. W poleceniu zawarliśmy informacje o produkcie, dla którego generujemy nazwy. Podajemy też podobny przykład, aby pokazać wzór, jaki chcemy uzyskać. Ustawiliśmy również wysoką wartość temperatury, aby zwiększyć losowość i uzyskać bardziej innowacyjne odpowiedzi.

Opis produktu: Domowy shaker do robienia koktajli mlecznych  
Słowa kluczowe: szybki, zdrowy, kompaktowy.  
Nazwy produktów: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Opis produktu: Para butów, które dopasowują się do każdego rozmiaru stopy.  
Słowa kluczowe: adaptacyjny, dopasowanie, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Odnośniki  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Przykłady OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Najlepsze praktyki dostrajania GPT-3 do klasyfikacji tekstu](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Po więcej pomocy  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Współtwórcy
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony przy użyciu usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić dokładność, należy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło wiążące. W przypadku informacji o kluczowym znaczeniu zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
